## Face detection in a video

### Import the modules

In [1]:
import cv2
import numpy as np

### Loading the pretrained HAAR Cascade

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [3]:
def rgb_to_ycgco(image):
    # Extracting R, G, and B channels
    R = image[:, :, 0]
    G = image[:, :, 1]
    B = image[:, :, 2]
    
    # Calculating Co, tmp, Cg, and Y
    Co = R - B
    tmp = B + Co / 2
    Cg = G - tmp
    Y = tmp + Cg / 2
    
    # Stacking Y, Cg, and Co channels to form YCGCO image
    ycgco_image = np.stack([Y, Cg, Co], axis=-1)
    
    return ycgco_image.astype(np.float32)

### Caputre the video

In [6]:
cap = cv2.VideoCapture('demo_V.mp4')
# Getting the frame rate of the video such that it will run on original speed 
fps = cap.get(cv2.CAP_PROP_FPS)
while True:
    # Capture frame by frame
    ret, frame = cap.read() # ret is return variable which is true when function has read something
    
    if ret is not True:
        break
    width = 400
    height = 500
    resized_frame = cv2.resize(frame, (width, height))

    # Convert the frame to grayscale
    ycgco_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2YCrCb)
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

     # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.rectangle(ycgco_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi1_x = x + w//8
        roi1_y = y + h//2
        roi1_w = w//10
        roi1_h = h//10
        cv2.rectangle(resized_frame, (roi1_x, roi1_y), (roi1_x+roi1_w, roi1_y+roi1_h), (255, 255, 0), 2)
        cv2.rectangle(ycgco_frame, (roi1_x, roi1_y), (roi1_x+roi1_w, roi1_y+roi1_h), (255, 255, 0), 2)

        roi2_x = x + 2*w//8
        roi2_y = y + 2*h//4
        roi2_w = w//10
        roi2_h = h//10
        cv2.rectangle(resized_frame, (roi2_x, roi2_y), (roi2_x+roi2_w, roi2_y+roi2_h), (255, 255, 255), 2)
        cv2.rectangle(ycgco_frame, (roi2_x, roi2_y), (roi2_x+roi2_w, roi2_y+roi2_h), (255, 255, 255), 2)
        
    # Show the output
    cv2.imshow('Face detection frame', ycgco_frame)
    # Calculate delay time based on frame rate
    delay_time = int(1000 / fps)  # Convert from fps to milliseconds
    # Check for 'q' key to exit
    key = cv2.waitKey(delay_time) & 0xFF
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()